# 📌 2. rag/chunker.py

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=40,
    separators=["\n\n", "\n", ".", "?", "!"]
)


def chunk_field(text: str) -> list:
    if not text:
        return []
    return text_splitter.split_text(text)


def build_corpus(patient: dict) -> list:
    docs = []

    summary_text = (
        f"Name: {patient.get('name')}. "
        f"Age/Sex: {patient.get('age')}/{patient.get('sex')}. "
        f"Conditions: {', '.join(patient.get('conditions', []))}."
    )

    for chunk in chunk_field(summary_text):
        docs.append({
            "text": chunk,
            "source": "summary",
            "meta": {"patient_id": patient["id"]}
        })

    for chunk in chunk_field(patient.get("care_plan", "")):
        docs.append({
            "text": chunk,
            "source": "care_plan",
            "meta": {"patient_id": patient["id"]}
        })

    for event in patient.get("timeline", []):
        txt = f"{event['date']}: {event['text']}"
        for chunk in chunk_field(txt):
            docs.append({
                "text": chunk,
                "source": "timeline",
                "meta": {"patient_id": patient["id"], "date": event["date"]}
            })

    for chunk in chunk_field(patient.get("notes", "")):
        docs.append({
            "text": chunk,
            "source": "notes",
            "meta": {"patient_id": patient["id"]}
        })

    return docs


# 📌 3. rag/embedder.py

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json

MODEL_NAME = "all-MiniLM-L6-v2"
_cache_dir = Path(".cache")
_cache_dir.mkdir(exist_ok=True)

_model = None


def get_model():
    global _model
    if _model is None:
        _model = SentenceTransformer(MODEL_NAME)
    return _model


def embed_texts(texts, patient_id: str):
    cache_e = _cache_dir / f"{patient_id}_embeddings.npy"
    cache_meta = _cache_dir / f"{patient_id}_meta.json"

    if cache_e.exists() and cache_meta.exists():
        meta = json.loads(cache_meta.read_text())
        if meta.get("n_texts") == len(texts):
            return np.load(cache_e)

    model = get_model()
    embs = model.encode(texts, convert_to_numpy=True)

    np.save(cache_e, embs)
    cache_meta.write_text(json.dumps({"n_texts": len(texts)}))

    return embs


# 📌 4. rag/retriever.py

In [7]:
import numpy as np
from typing import List, Tuple, Dict
from sklearn.metrics.pairwise import cosine_similarity


def retrieve(
    query_vector: np.ndarray,
    docs: List[Dict],
    doc_vectors: np.ndarray,
    top_k: int = 3,
    filter_source: str = None
) -> List[Tuple[Dict, float]]:
    if filter_source:
        filtered_docs = []
        filtered_vectors = []
        for i, d in enumerate(docs):
            if d["source"] == filter_source:
                filtered_docs.append(d)
                filtered_vectors.append(doc_vectors[i])
        if filtered_docs:
            docs = filtered_docs
            doc_vectors = np.array(filtered_vectors)

    scores = cosine_similarity([query_vector], doc_vectors)[0]
    top_idx = scores.argsort()[-top_k:][::-1]

    return [(docs[i], float(scores[i])) for i in top_idx]


# 📌 5. rag/router.py

In [8]:
import re

def route_question(question: str) -> str:
    q = question.lower()

    if re.search(r"\b(when|date|recent|time|happened|follow|appointment)\b", q):
        return "timeline"

    if re.search(r"\b(plan|med|dose|treatment|instruction)\b", q):
        return "care_plan"

    if re.search(r"\b(notes|symptom|issue|problem|feeling|status)\b", q):
        return "notes"

    if re.search(r"\b(condition|summar|overview|profile)\b", q):
        return "summary"

    return "all"


# 📌 6. rag/answer_generator.py

In [9]:
import os
from typing import List, Tuple, Dict
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

load_dotenv()

HF_API_KEY = os.environ.get("HF_API_KEY")
HF_MODEL = os.environ.get("HF_MODEL", "openai/gpt-oss-20b")

client = None
if HF_API_KEY:
    try:
        client = InferenceClient(HF_MODEL, token=HF_API_KEY)
    except Exception:
        client = None


def build_prompt(question: str, retrieved_chunks: List[Tuple[Dict, float]]) -> str:
    system = (
        "You are a clinical assistant. Answer ONLY using the patient context provided. "
        "If information is missing, say 'Information not found in patient record.' "
        "Keep answers concise (1–3 sentences)."
    )

    context = "\n\n".join(
        f"[{doc['source']}] {doc['text']}"
        for doc, _ in retrieved_chunks
    ) if retrieved_chunks else "No context available."

    return (
        f"{system}\n\n"
        f"Patient context:\n{context}\n\n"
        f"Question: {question}\n"
        "Answer:"
    )


def call_hf_llm(prompt: str) -> str:
    if not client:
        return None

    try:
        output = client.text_generation(prompt, max_new_tokens=200, temperature=0.1)
        return output.strip()
    except Exception:
        return None


def fallback_answer(question: str, retrieved_chunks, patient):
    q = question.lower()

    if "condition" in q or "summar" in q:
        conds = ", ".join(patient.get("conditions", []))
        return f"Conditions: {conds}" if conds else "Information not found in patient record."

    if "when" in q or "review" in q or "date" in q:
        for doc, _ in retrieved_chunks:
            date = doc["meta"].get("date")
            if date:
                return f"Relevant timeline date: {date}. {doc['text']}"
        return "Information not found in patient record."

    if "recent" in q or "happened" in q:
        if retrieved_chunks:
            doc, _ = retrieved_chunks[0]
            return f"Most recent entry: {doc['text']}"
        return "Information not found in patient record."

    if retrieved_chunks:
        return " ".join(doc["text"] for doc, _ in retrieved_chunks)

    return "No relevant information found."


def generate_answer(question, retrieved_chunks, route, patient_meta):
    prompt = build_prompt(question, retrieved_chunks)
    llm_out = call_hf_llm(prompt)

    if llm_out:
        return llm_out

    return fallback_answer(question, retrieved_chunks, patient_meta)


# 📌 1. chatbot.py

In [10]:
#!/usr/bin/env python3
import argparse
import json
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer

# from rag.chunker import build_corpus
# from rag.embedder import embed_texts, get_model
# from rag.retriever import retrieve
# from rag.router import route_question
# from rag.answer_generator import generate_answer


def load_patient(patient_id: str):
    pfile = Path("patients") / f"{patient_id}.json"
    if not pfile.exists():
        raise FileNotFoundError(f"Patient file not found: {pfile}")
    return json.loads(pfile.read_text(encoding="utf-8"))



def prepare_patient(patient: dict):
    pid = patient["id"]

    docs = build_corpus(patient)
    texts = [d["text"] for d in docs]
    vectors = embed_texts(texts, pid)

    return docs, vectors


In [11]:
patient_id = "P001"

In [12]:
patient = load_patient(patient_id)
patient

{'id': 'P001',
 'name': 'Aarti Sharma',
 'age': 32,
 'sex': 'F',
 'conditions': ['Gestational Diabetes', 'Iron-deficiency Anemia'],
 'care_plan': '12-week antenatal care: weekly fasting and post-prandial glucose monitoring; start oral iron supplement (60mg elemental) daily; nutrition counseling and follow-up. Next review scheduled on 2025-12-10 for glycemic control assessment.',
 'timeline': [{'date': '2025-11-20',
   'text': 'First antenatal visit; baseline labs collected.'},
  {'date': '2025-11-25',
   'text': 'Nutrition counselling given; advised diet changes.'},
  {'date': '2025-12-01',
   'text': 'Fasting glucose elevated; advised stricter monitoring.'}],
 'notes': 'Patient reports fatigue and mild dizziness. Adherent to diet but occasional missed doses of iron. Will monitor BP and glucose at home.'}

In [ ]:
docs, doc_vectors = prepare_patient(patient)
docs, doc_vectors

([{'text': 'Name: Aarti Sharma. Age/Sex: 32/F. Conditions: Gestational Diabetes, Iron-deficiency Anemia.',
   'source': 'summary',
   'meta': {'patient_id': 'P001'}},
  {'text': '12-week antenatal care: weekly fasting and post-prandial glucose monitoring; start oral iron supplement (60mg elemental) daily; nutrition counseling and follow-up. Next review scheduled on 2025-12-10 for glycemic control assessment.',
   'source': 'care_plan',
   'meta': {'patient_id': 'P001'}},
  {'text': '2025-11-20: First antenatal visit; baseline labs collected.',
   'source': 'timeline',
   'meta': {'patient_id': 'P001', 'date': '2025-11-20'}},
  {'text': '2025-11-25: Nutrition counselling given; advised diet changes.',
   'source': 'timeline',
   'meta': {'patient_id': 'P001', 'date': '2025-11-25'}},
  {'text': '2025-12-01: Fasting glucose elevated; advised stricter monitoring.',
   'source': 'timeline',
   'meta': {'patient_id': 'P001', 'date': '2025-12-01'}},
  {'text': 'Patient reports fatigue and mil

In [ ]:
embedder = get_model()
while True:
    query = input("Q> ").strip()
    if query.lower() in ("exit", "quit"):
        print("bye")
        break

    route = route_question(query)
    print(route)
    query_vector = embedder.encode([query])[0]
    print(query_vector)

    retrieved = retrieve(
        query_vector=query_vector,
        docs=docs,
        doc_vectors=doc_vectors,
        top_k=3,
        filter_source=route
    )
    print(retrieved)
    print("\n")

    answer = generate_answer(
        question=query,
        retrieved_chunks=retrieved,
        route=route,
        patient_meta=patient
    )
    print(answer)

    print("\n--- Retrieved Context ---")
    for doc, score in retrieved:
        print(f"[{doc['source']}] score={score:.4f} | {doc['text']}")

    print("\n--- Answer ---")
    print(answer)
    print("\n-------------------------\n")


In [ ]:


def interactive_chat(patient_id: str):
    patient = load_patient(patient_id)
    docs, doc_vectors = prepare_patient(patient)
    embedder = get_model()

    print(f"\nLoaded patient {patient_id}: {patient.get('name')}")
    print(f"Number of chunks: {len(docs)}")
    print("Type 'exit' or 'quit' to stop.\n")

    while True:
        query = input("Q> ").strip()
        if query.lower() in ("exit", "quit"):
            print("bye")
            break

        route = route_question(query)
        query_vector = embedder.encode([query])[0]

        retrieved = retrieve(
            query_vector=query_vector,
            docs=docs,
            doc_vectors=doc_vectors,
            top_k=3,
            filter_source=route
        )


        answer = generate_answer(
            question=query,
            retrieved_chunks=retrieved,
            route=route,
            patient_meta=patient
        )

        print("\n--- Retrieved Context ---")
        for doc, score in retrieved:
            print(f"[{doc['source']}] score={score:.4f} | {doc['text']}")

        print("\n--- Answer ---")
        print(answer)
        print("\n-------------------------\n")


# def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--patient", required=True,
#                         help="Patient ID (P001, P002...)")
#     args = parser.parse_args()

#     pid = args.patient.upper()
#     interactive_chat(pid)


# if __name__ == "__main__":
#     main()
